In [1]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import scipy
from skimage.color import gray2rgb
from tqdm import tqdm
import tifffile
from PIL import Image
import json
import matplotlib.pyplot as plt

In [2]:
adata = sc.read("xenium_outs/merged_processed.h5ad")

In [3]:
adata.obs["biopsy_ID"] = np.nan

## add biopsy IDs

In [4]:
folder = "cleaning_annots/biopsies/"
for e in os.listdir(folder):
    if "." not in e:
        sub = adata[adata.obs["Slide_ID"]==e]
        sub_folder = os.path.join(folder, e)
        for f in os.listdir(sub_folder):
            if ".csv" in f:
                name = f.split(".csv")[0]
                coords = pd.read_csv(os.path.join(sub_folder, f), index_col=0)[["axis-1", "axis-0"]]
                for annot in coords.index.unique():
                    tmp = coords.loc[annot]
                    polygon = np.array(tmp)
                    points = np.array(sub.obsm["spatial"])
                    tupVerts=[]
                    for i in range(polygon.shape[0]):
                        tupVerts.append(tuple(polygon[i,:]))
                    poly = plt.Polygon(tupVerts, ec="k")
                    grid = poly.contains_points(points)
                    idxs = sub.obs.index[grid]
                    adata.obs.loc[idxs, "biopsy_ID"] = f"{e} - X{name}"

/tmp/ipykernel_44861/3237762282.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0018775 - X2' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  adata.obs.loc[idxs, "biopsy_ID"] = f"{e} - X{name}"


## folds

In [5]:
adata.obs["is_fold"] = "no"

In [6]:
folder = "cleaning_annots/fold_annots"
for e in os.listdir(folder):
    if "." not in e:
        sub = adata[adata.obs["Slide_ID"]==e]
        sub_folder = os.path.join(folder, e)
        for f in os.listdir(sub_folder):
            if ".csv" in f:
                name = f.split(".csv")[0]
                coords = pd.read_csv(os.path.join(sub_folder, f), index_col=0)[["axis-1", "axis-0"]]
                for annot in coords.index.unique():
                    tmp = coords.loc[annot]
                    polygon = np.array(tmp)
                    points = np.array(sub.obsm["spatial"])
                    tupVerts=[]
                    for i in range(polygon.shape[0]):
                        tupVerts.append(tuple(polygon[i,:]))
                    poly = plt.Polygon(tupVerts, ec="k")
                    grid = poly.contains_points(points)
                    idxs = sub.obs.index[grid]
                    adata.obs.loc[idxs, "is_fold"] = "yes"

In [7]:
adata.obs["is_fold"] = adata.obs["is_fold"].astype("category").cat.reorder_categories(["no", "yes"])
adata.uns["is_fold_colors"] = ["Black", "white"]

## blurs

In [8]:
adata.obs["is_blur"] = "no"

In [9]:
folder = "cleaning_annots/blurs_annots"
for e in os.listdir(folder):
    if "." not in e:
        sub = adata[adata.obs["Slide_ID"]==e]
        sub_folder = os.path.join(folder, e)
        for f in os.listdir(sub_folder):
            if ".csv" in f:
                name = f.split(".csv")[0]
                coords = pd.read_csv(os.path.join(sub_folder, f), index_col=0)[["axis-1", "axis-0"]]
                for annot in coords.index.unique():
                    tmp = coords.loc[annot]
                    polygon = np.array(tmp)
                    points = np.array(sub.obsm["spatial"])
                    tupVerts=[]
                    for i in range(polygon.shape[0]):
                        tupVerts.append(tuple(polygon[i,:]))
                    poly = plt.Polygon(tupVerts, ec="k")
                    grid = poly.contains_points(points)
                    idxs = sub.obs.index[grid]
                    adata.obs.loc[idxs, "is_blur"] = "yes"

In [10]:
adata.obs["is_blur"] = adata.obs["is_blur"].astype("category").cat.reorder_categories(["no", "yes"])
adata.uns["is_blur_colors"] = ["Black", "white"]

In [11]:
for key in sorted(adata.obs.Slide_ID.unique()):
    
    sub = adata[adata.obs.Slide_ID==key]
    if "yes" in sub.obs["is_fold"].unique():
        print(key)

0011284
0011287
0011762


In [12]:
for key in sorted(adata.obs.Slide_ID.unique()):
    sub = adata[adata.obs.Slide_ID==key]
    if "yes" in sub.obs["is_blur"].unique():
        print(key)

0011284
0011287
0011546
0011695
0011707
0011762
0018775


-----

In [26]:
figdir = "spatial_plots_biopsies"
if not os.path.exists(figdir):
    os.mkdir(figdir)

In [27]:
sc.set_figure_params(dpi_save=200)

In [28]:
LEVEL = 1

In [29]:
for key in tqdm(sorted(adata.obs.Slide_ID.unique())):
    titles = ["DAPI", "biopsy_ID"]
    colors = [None, "biopsy_ID"]
    legend_locs = [None, "right margin"]
    plt.figure(figsize=(10,10))
    for i in range(1,3):
        ax = plt.subplot(1,2,i)
        sc.pl.spatial(adata[adata.obs.Slide_ID==key],
                      img_key=f"dapi_LEVEL{LEVEL}",
                      library_id=key,
                      title=titles[i-1],
                      color=colors[i-1], show=False, frameon=False, legend_loc=legend_locs[i-1], ax=ax)
    plt.tight_layout()
    plt.savefig(os.path.join(figdir, f"{key}.pdf"), bbox_inches="tight")
    plt.close()

  0%|          | 0/8 [00:00<?, ?it/s]/opt/conda/envs/xenium_cgn/lib/python3.11/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/opt/conda/envs/xenium_cgn/lib/python3.11/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
 12%|█▎        | 1/8 [05:11<36:19, 311.41s/it]/opt/conda/envs/xenium_cgn/lib/python3.11/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/opt/conda/envs/xenium_cgn/lib/python3.11/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
 25%|██▌       | 2/8 [09:03<26:29, 264.92s/it]/opt/conda/envs/xenium_cgn/lib/python3.11/site-packages/anndata

-----

In [21]:
figdir = "spatial_plots_folds"
if not os.path.exists(figdir):
    os.mkdir(figdir)

In [22]:
sc.set_figure_params(dpi_save=200)

In [23]:
LEVEL = 1

In [25]:
for key in tqdm(sorted(adata.obs.Slide_ID.unique())):
    titles = ["DAPI", "is_fold"]
    colors = [None, "is_fold"]
    legend_locs = [None, "right margin"]
    plt.figure(figsize=(10,10))
    for i in range(1,3):
        ax = plt.subplot(1,2,i)
        sc.pl.spatial(adata[adata.obs.Slide_ID==key],
                      img_key=f"dapi_LEVEL{LEVEL}",
                      library_id=key,
                      title=titles[i-1],
                      color=colors[i-1], show=False, frameon=False, legend_loc=legend_locs[i-1], ax=ax)
    plt.tight_layout()
    plt.savefig(os.path.join(figdir, f"{key}.pdf"), bbox_inches="tight")
    plt.close()

  0%|          | 0/8 [00:00<?, ?it/s]/opt/conda/envs/xenium_cgn/lib/python3.11/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/opt/conda/envs/xenium_cgn/lib/python3.11/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
 12%|█▎        | 1/8 [05:22<37:40, 322.87s/it]/opt/conda/envs/xenium_cgn/lib/python3.11/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/opt/conda/envs/xenium_cgn/lib/python3.11/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
 25%|██▌       | 2/8 [09:23<27:28, 274.67s/it]/opt/conda/envs/xenium_cgn/lib/python3.11/site-packages/anndata

-----

In [30]:
figdir = "spatial_plots_blurs"
if not os.path.exists(figdir):
    os.mkdir(figdir)

In [31]:
sc.set_figure_params(dpi_save=200)

In [32]:
LEVEL = 1

In [33]:
for key in tqdm(sorted(adata.obs.Slide_ID.unique())):
    titles = ["DAPI", "is_blur"]
    colors = [None, "is_blur"]
    legend_locs = [None, "right margin"]
    plt.figure(figsize=(10,10))
    for i in range(1,3):
        ax = plt.subplot(1,2,i)
        sc.pl.spatial(adata[adata.obs.Slide_ID==key],
                      img_key=f"dapi_LEVEL{LEVEL}",
                      library_id=key,
                      title=titles[i-1],
                      color=colors[i-1], show=False, frameon=False, legend_loc=legend_locs[i-1], ax=ax)
    plt.tight_layout()
    plt.savefig(os.path.join(figdir, f"{key}.pdf"), bbox_inches="tight")
    plt.close()

  0%|          | 0/8 [00:00<?, ?it/s]/opt/conda/envs/xenium_cgn/lib/python3.11/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/opt/conda/envs/xenium_cgn/lib/python3.11/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
 12%|█▎        | 1/8 [05:27<38:12, 327.53s/it]/opt/conda/envs/xenium_cgn/lib/python3.11/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/opt/conda/envs/xenium_cgn/lib/python3.11/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
 25%|██▌       | 2/8 [09:01<26:03, 260.57s/it]/opt/conda/envs/xenium_cgn/lib/python3.11/site-packages/anndata

In [34]:
adata.write("xenium_outs/merged_processed_biopsy_ID_added.h5ad")

-----

In [13]:
adata_orig = sc.read("xenium_outs/merged_processed.h5ad")
adata_orig

AnnData object with n_obs × n_vars = 3230744 × 480
    obs: 'x', 'y', 'z', 'cluster', 'n_transcripts', 'density', 'elongation', 'area', 'avg_confidence', 'avg_assignment_confidence', 'max_cluster_frac', 'lifespan', 'x_centroid', 'y_centroid', 'cell_area', 'Slide_ID', 'batch', 'Patient_Sample_ID', 'Disease', 'n_genes', 'celltype_l1', 'celltype_l1_codes', 'celltype_l1_prob'
    var: 'gene_ids', 'feature_types'
    uns: 'celltype_l1_colors', 'log1p', 'neighbors', 'pca', 'spatial', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap', 'spatial'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [14]:
print((adata_orig.obs.index==adata.obs.index).all())

True


In [15]:
adata_orig.obs["Biopsy_ID"] = adata.obs["biopsy_ID"].tolist()
adata_orig.obs["Biopsy_ID"] = adata_orig.obs["Biopsy_ID"].astype("category")

In [16]:
keep_idxs = adata[(adata.obs["is_fold"]=="no")&(adata.obs["is_blur"]=="no")].obs.index
print(len(keep_idxs))

3222037


In [17]:
print(f"{len(keep_idxs)} out of {adata_orig.shape[0]} cells kept.")

3222037 out of 3230744 cells kept.


In [18]:
keep_idxs

Index(['cell_CRfb095a77d-1', 'cell_CRfb095a77d-2', 'cell_CRfb095a77d-3',
       'cell_CRfb095a77d-5', 'cell_CRfb095a77d-6', 'cell_CRfb095a77d-7',
       'cell_CRfb095a77d-8', 'cell_CRfb095a77d-9', 'cell_CRfb095a77d-10',
       'cell_CRfb095a77d-11',
       ...
       'cell_CRc71d71060-597345', 'cell_CRc71d71060-597482',
       'cell_CRc71d71060-597501', 'cell_CRc71d71060-597622',
       'cell_CRc71d71060-597872', 'cell_CRc71d71060-597925',
       'cell_CRc71d71060-598041', 'cell_CRc71d71060-598270',
       'cell_CRc71d71060-598508', 'cell_CRc71d71060-598872'],
      dtype='object', length=3222037)

In [19]:
adata_orig = adata_orig[keep_idxs]

In [20]:
adata_orig.write("xenium_outs/merged_processed_cleaned.h5ad")